In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import os

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
study_name = COMBINE_harmonizer.STUDY_OC
sheet_name = COMBINE_harmonizer.SHEET_DERIVED_DATA

root_dir = '..'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

## 02. Dictionary

In [4]:
data_dict_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'

In [5]:
df_dict = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name)

In [6]:
dict_columns = list(df_dict[study_name].unique())

## 03. OC

In [7]:
the_dir = cfg.config[f'{study_name}_analysis_dir']

In [8]:
filenames = os.listdir(the_dir)
filenames.sort()

In [9]:
dfs = [pd.read_csv(os.sep.join([the_dir, each_filename])) for each_filename in filenames]

### 03.1. check columns in each file

In [10]:
for idx, each_df in enumerate(dfs):
    print(f'({idx}/{len(dfs)}) filename: {filenames[idx]} columns: {list(each_df.columns)}')

(0/2) filename: analysis.csv columns: ['HTHRM_ID', 'CENTER', 'ocfolnum', 'fcenter', 'trt_dur', 'trt_temp', 'hie_fix', 'blindness', 'hearing', 'hear_imp', 'gross', 'cp_out', 'modsevcp', 'bayley3', 'b3_lang', 'b3_motor', 'mult_imp', 'death18', 'gas_tube', 'all_norm', 'adjudicate', 'age_death', 'apg10_lt5', 'mom_ed', 'anticon_0', 'inot_0', 'any_inot', 'ecsect', 'male', 'apgar5_5', 'apgar10_5', 'age_rand', 'hosp_die', 'any_antic', 'any_seiz', 'seiz_fu', 'disab_ms', 'disab_4', 'disab_die', 'disab_die4', 'other_cere', 'edema', 'use_od', 'use_pi', 'out_grp', 'use_MRIage', 'use_hemi', 'frontal', 'pariet', 'tempo', 'occip', 'cereb_any', 'fp_max', 'pt_max', 'to_max', 'para_max', 'per_max', 'peri_max', 'ins_max', 'vent_max', 'bel_max', 'cr_max', 'hip_max', 'bs_max', 'pit_max', 'hy_max', 'op_max', 'ex_max', 's_max', 'bg_max', 't_max', 'cc_max', 'vas_max', 'oth_max', 'po_max', 'use_infarc', 'use_atrophy', 'use_global', 'use_ccthin', 'use_vd', 'new_bgt', 'new_plic', 'new_water', 'new_wmi', 'extent_n

### 03.2. all columns

In [11]:
all_columns = []
for idx, each_df in enumerate(dfs):
    all_columns += list(map(lambda x: {'column': x, 'idx': idx, 'filename': filenames[idx]}, list(each_df.columns)))
df_columns = pd.DataFrame(all_columns)
df_columns['column:filename'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}", axis=1)
df_columns['column:filename:idx'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}:{x['idx']}", axis=1)
df_columns.sort_values(by=['filename', 'column'], inplace=True)

In [12]:
df_columns

,column,idx,filename,column:filename,column:filename:idx
1,CENTER,0,analysis.csv,CENTER:analysis.csv,CENTER:analysis.csv:0
0,HTHRM_ID,0,analysis.csv,HTHRM_ID:analysis.csv,HTHRM_ID:analysis.csv:0
92,MRI_2cat,0,analysis.csv,MRI_2cat:analysis.csv,MRI_2cat:analysis.csv:0
94,acidosis,0,analysis.csv,acidosis:analysis.csv,acidosis:analysis.csv:0
20,adjudicate,0,analysis.csv,adjudicate:analysis.csv,adjudicate:analysis.csv:0
...,...,...,...,...,...
144,use_pi,1,ocout.csv,use_pi:ocout.csv,use_pi:ocout.csv:1
180,use_vd,1,ocout.csv,use_vd:ocout.csv,use_vd:ocout.csv:1
196,usualc,1,ocout.csv,usualc:ocout.csv,usualc:ocout.csv:1
173,vas_max,1,ocout.csv,vas_max:ocout.csv,vas_max:ocout.csv:1


In [13]:
df_columns_count = df_columns.groupby(['column']).agg(count=('column', 'count')).sort_values(by=['count'], ascending=False)
df_columns_count.head(20)

,count
column,
CENTER,2
new_wmi,2
peri_max,2
per_max,2
pariet,2
para_max,2
out_grp,2
other_cere,2
oth_max,2


#### column not in dict

In [14]:
is_not_in_dict = df_columns['column'].isin(dict_columns) == False
df_columns_not_in_dict = df_columns[is_not_in_dict]
df_columns_not_in_dict

,column,idx,filename,column:filename,column:filename:idx


In [15]:
columns_not_in_dict = list(df_columns_not_in_dict['column:filename:idx'].unique())
# columns_not_in_dict.sort()
columns_not_in_dict

[]

#### dict not in column

In [16]:
is_dict_not_in_columns = (df_dict['OC'].isin(df_columns['column']) == False) & (df_dict['OC'].isnull() == False)
df_dict_not_in_columns = df_dict[is_dict_not_in_columns]
df_dict_not_in_columns

,Category,Subcategory,Standardized_VariableNames_Dictionary,type,Variable_Description,#studies w/ this var,redcap,comment,LH,OC


## 04. show column values

In [17]:
# XXX no show due to data privacy concern.

'''
for idx, each_df in enumerate(dfs):
    for idx2, column in enumerate(each_df.columns):
        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')
'''

"\nfor idx, each_df in enumerate(dfs):\n    for idx2, column in enumerate(each_df.columns):\n        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')\n"